# Modified TPC-H Q1 Microbenchmark
This file is adapted to use numpy only without Spark.

```sql
select
  sum(l_quantity) as sum_qty,
  sum(l_extendedprice) as sum_base_price,
  sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
  sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
  avg(l_quantity) as avg_qty,
  avg(l_extendedprice) as avg_price,
  avg(l_discount) as avg_disc,
  count(*) as count_order
from
  lineitem
where
  l_shipdate <= '1998-09-02'
  and l_returnflag = 'A'
  and l_linestatus = 'F'
```

In [1]:
EXPERIMENT = 'pac-q1-svd'
OUTPUT_DIR = f'./outputs/{EXPERIMENT}'
GENERATE = True
USE_EVEN_NUMBER_OF_INPUT_ROWS = False
SEED_RANDOM_NUMBER_GENERATOR = True

SAMPLING_METHOD = 'poisson' # 'poisson' or 'half'

if GENERATE:
    print("GENERATE = True, so we will generate new samples.")
else:
    print("GENERATE = False, so we will load saved output from files rather than recomputing.")

import os
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)


GENERATE = True, so we will generate new samples.


### Running PAC

In [2]:
from typing import Any, Callable, Dict, List, Tuple, Union

import numpy as np
if SEED_RANDOM_NUMBER_GENERATOR:
    np.random.seed(0)

import pandas as pd
from pandas import DataFrame
import concurrent.futures
import pyarrow as pa
import pyarrow.parquet as pq
import pickle
from numpy.random import laplace
from functools import reduce
import operator
from IPython.display import display, HTML
from datetime import date
from scipy import special

In [3]:
### Mayuri's conversion functions between DP epsilon and PAC MI using posterior advantage for equivalence
def calc_posterior(mi, prior=0.5, prec = 100000):
    test_vals = [x / prec for x in range(1, prec)]
    max_t = None
    for t in test_vals:
        if t*np.log(t/prior)+(1-t)*np.log((1-t)/(1-prior)) <= mi:
            if  max_t is None or t > max_t:
                max_t = t
    return max_t

def dp_epsilon_to_posterior_success(epsilon):
    return 1 - 1./(1+np.exp(epsilon))

def dp_ps_to_epsilon(ps):
    return np.log(ps / (1-ps))

# example usage:
# dp_ps_to_epsilon(calc_posterior(1/256.))

In [4]:
### Data Setup
#por_df = pq.read_table(f"./data/student_performance/student-por.parquet").to_pandas()
lineitem_df = pd.read_parquet('data/tpch/lineitem.parquet')

lineitem_df.shape

(600572, 16)

In [5]:
type(lineitem_df['l_linestatus'][0])

str

In [6]:
def runquery(lineitem_df: DataFrame) -> int:
    # 1. Filter lineitem rows where commit date is before receipt date.
    lineitem_filtered = lineitem_df[
        (lineitem_df['l_shipdate'] <= date(1998, 9, 2)) &
        (lineitem_df['l_returnflag'] == 'A') &
        (lineitem_df['l_linestatus'] == 'F')
    ]
    print(lineitem_filtered.shape)

    # 2. Pre-compute
    discounted_price = lineitem_filtered['l_extendedprice'] * (1 - lineitem_filtered['l_discount'])
    charged_price = discounted_price * (1 + lineitem_filtered['l_tax'])

    # 3. Get aggregations
    aggregated_result = [
        float(lineitem_filtered['l_quantity'].sum()),       # sum_qty
        float(lineitem_filtered['l_extendedprice'].sum()),  # sum_base_price
        float(discounted_price.sum()),                      # sum_disc_price
        float(charged_price.sum()),                         # sum_charge
        float(lineitem_filtered['l_quantity'].mean()),      # avg_qty
        float(lineitem_filtered['l_extendedprice'].mean()), # avg_price
        float(lineitem_filtered['l_discount'].mean()),      # avg_disc
        float(len(lineitem_filtered))                       # count_order (faster than .count())
    ]

    # print(aggregated_result)

    return aggregated_result
runquery(lineitem_df)

(147790, 16)


[3774200.0,
 5320753880.69,
 5054096266.6828,
 5256751331.449234,
 25.537587116854997,
 36002.12382901414,
 0.05014459706340077,
 147790.0]

In [7]:
### Query Setup
SAMPLES = 100
assert SAMPLES % 2 == 0, "We need an even number of samples for paired sampling."
number_of_pairs = SAMPLES // 2

OUTPUT_COLS = ['sum_qty', 'sum_base_price', 'sum_disc_price', 'sum_charge', 'avg_qty', 'avg_price', 'avg_disc', 'count_order']

true_result = np.array(runquery(lineitem_df)) # Save the true result of the query for later
#true_result = np.divide(true_result, 2) # manually correct count = count * 2

number_of_contributing_rows = 147790  # hardcode number of eligible rows

def poisson_paired_sample(df: DataFrame) -> Tuple[DataFrame, DataFrame]:
    """
    This will select a subset of indices, where each index is selected with probability 0.5.
    The first result is the dataframe composed of the selected rows.
    The second result is the complement / the dataframe composed of the rows that were not selected.
    """
    mask = np.random.random_sample(len(df)) < 0.5  # Generates a bitmask of length df.shape[0] where each bit is 1 with probability 0.5
    selected = df[mask]
    not_selected = df[~mask]
    return selected, not_selected

def half_paired_sample(df: DataFrame) -> Tuple[DataFrame, DataFrame]:
    """
    This will select half of the row indices from the dataframe at random.
    The first result is the dataframe composed of the selected rows.
    The second result is the complement / the dataframe composed of the rows that were not selected.
    """
    indices = np.random.choice(df.index, size=(df.shape[0] // 2), replace=False)
    not_indices = list(set(df.index) - set(indices))
    selected: DataFrame = df.loc[indices]
    not_selected: DataFrame = df.loc[not_indices]
    return (selected, not_selected)

def sample_using_chosen_method(df: DataFrame) -> Tuple[DataFrame, DataFrame]:
    if SAMPLING_METHOD == 'poisson':
        return poisson_paired_sample(df)
    elif SAMPLING_METHOD == 'half':
        return half_paired_sample(df)

def generate_samples(laplace_lambda: float = 1.0, alpha: int = 10) -> List[np.ndarray]:
    # Apply thresholding: if there are less than alpha samples
    # (with some Laplacian noise), then we don't subsample and return None.
    laplace_noise: float = np.random.laplace(scale=laplace_lambda)

    if number_of_contributing_rows + laplace_noise < alpha:  # if we don't get enough results from the query
        print("There are not enough rows contributing to the result for PAC to be meaningful.")
        return []
    
    number_of_pairs = SAMPLES // 2
    out_np: List[np.ndarray] = []
    for i in range(number_of_pairs):
        print(f'iteration : {i}')
        for temp_df in sample_using_chosen_method(lineitem_df.reset_index(drop=True)):  # reset index to sequential
            out = runquery(temp_df)
            # double the sums and counts, avg stays the same
            out[0] *= 2
            out[1] *= 2
            out[2] *= 2
            out[3] *= 2
            out[7] *= 2
            # for a 2d array, flatten it
            out_np.append(np.array(out))
    # Debug: SVD new (not needed I think??)
    # out_np = reduce(operator.iconcat, out_np, [])

    return out_np

(147790, 16)


In [8]:
out_np = generate_samples()

iteration : 0
(73684, 16)
(74106, 16)
iteration : 1
(73818, 16)
(73972, 16)
iteration : 2
(73858, 16)
(73932, 16)
iteration : 3
(74007, 16)
(73783, 16)
iteration : 4
(73832, 16)
(73958, 16)
iteration : 5
(73949, 16)
(73841, 16)
iteration : 6
(73827, 16)
(73963, 16)
iteration : 7
(73876, 16)
(73914, 16)
iteration : 8
(73943, 16)
(73847, 16)
iteration : 9
(74144, 16)
(73646, 16)
iteration : 10
(73870, 16)
(73920, 16)
iteration : 11
(73681, 16)
(74109, 16)
iteration : 12
(73801, 16)
(73989, 16)
iteration : 13
(73773, 16)
(74017, 16)
iteration : 14
(73555, 16)
(74235, 16)
iteration : 15
(73962, 16)
(73828, 16)
iteration : 16
(74054, 16)
(73736, 16)
iteration : 17
(73704, 16)
(74086, 16)
iteration : 18
(73663, 16)
(74127, 16)
iteration : 19
(73809, 16)
(73981, 16)
iteration : 20
(73772, 16)
(74018, 16)
iteration : 21
(74007, 16)
(73783, 16)
iteration : 22
(74058, 16)
(73732, 16)
iteration : 23
(73756, 16)
(74034, 16)
iteration : 24
(74167, 16)
(73623, 16)
iteration : 25
(74109, 16)
(73681, 

In [9]:
dimensions: int = len(out_np[0])

In [10]:
out_np_2darr = [np.atleast_2d(o) for o in out_np] # make sure all the DF -> np.ndarray conversions result in 2d arrays
est_y: np.ndarray = np.stack(out_np_2darr, axis=-1).reshape(dimensions, len(out_np))# shape (dimensions, samples)
    
est_y_for_proj = est_y.T

In [11]:
proj_var, u_trans  = np.linalg.eig(np.cov(est_y))

sqrt_total_var_svd = np.sum(np.sqrt(proj_var))

In [12]:
est_y.shape

(8, 100)

In [13]:
proj_t = []
for t in est_y_for_proj:
    proj_t.append(np.matmul(t, u_trans))
proj_t = np.array(proj_t)

In [14]:
sqrt_total_var_svd = np.sum(np.sqrt(np.var(proj_t)))
pac_noise_svd: np.ndarray = (1./(2*1)) * sqrt_total_var_svd * np.sqrt(np.var(proj_t, axis=0))


In [15]:
print(sqrt_total_var_svd)

2986359235.5046225


In [16]:
pac_noises_to_add_svd: np.ndarray = np.random.normal(loc=0, scale=np.sqrt(pac_noise_svd))


In [17]:
pac_noises_to_add_svd

array([ 1.59030146e+08, -2.76042428e+06,  1.36031775e+07, -1.81768346e+06,
       -6.30290100e+05, -1.71009273e+04, -2.02568149e+00, -3.83606832e+02])

In [18]:
sample = est_y[:,0]


In [19]:
sample

array([3.76986800e+06, 5.31825915e+09, 5.05263840e+09, 5.25522537e+09,
       2.55813202e+01, 3.60882902e+04, 5.00074643e-02, 1.47368000e+05])

In [20]:
proj = np.matmul(sample, u_trans)
proj += pac_noises_to_add_svd
proj = np.matmul(proj, u_trans.T)

In [21]:
proj

array([ 1.89590572e+06,  5.22464352e+09,  4.95366501e+09,  5.17201884e+09,
        1.40906480e+02,  2.19300459e+05,  3.83698269e+02, -5.35066681e+05])